In [ ]:
import itertools
import pandas as pd
import re
import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
def columnas(valores,variable):
    y = [re.findall("\\d+", j)[0] for j in valores]
    y = [int(i) for i in y]
    todas = list(set(y))
    df = pd.DataFrame()
    df[f"y_{variable}"] = y
    for k in todas:
        #print(k)
        df[f"{variable}_{k}"] = ([1 if k == j else 0 for j in y])
    return df

def matrix_X(df_triangulo):
    k = len(df_triangulo.columns)
    alpha = [f'a_{i}' for i in range(1,k+1)]
    mu    = [f'u_{i}' for i in range(1,k+1)]
    lists = [alpha, mu]
    df    = pd.DataFrame(list(itertools.product(*lists)), columns=['a', 'u'])

    alpha    = columnas(valores  = df.a, variable = 'a')
    mu       = columnas(valores=df.u, variable = 'u')
    df_col= pd.concat([alpha, mu], axis=1)


    df_col['y_a'] = df_col['y_a'].astype(str) + df_col['y_u'].astype(str) 
    df_col['y_a'] = [int(i) for i in df_col['y_a']]
    df_col = df_col.drop(['y_u', 'u_1'], axis=1)
    df_col['a_1'] = 1
    df_col.rename(columns={'a_1': 'b0'}, inplace=True)
    df_col.rename(columns={'y_a': 'y_ii'}, inplace=True)
    #df_col = df_col.drop(['y_ii'], axis=1)
    return df_col


def matrix_y(df_triangulo):
    k = len(df_triangulo.columns)
    d0 = pd.DataFrame()
    for i in range(k):
        for j in range(k):
            d1 = pd.DataFrame({'y_ii': [int(f'{i+1}{j+1}')], 'Y': [math.log(df_triangulo.iloc[i, j])]})
            d0 = pd.concat([d0, d1], axis=0)
    return d0

def triangulo(df, grcode, entreno):
    
    if entreno:
        df_trinagulo = df[(df['GRCODE']== grcode ) & (df['DevelopmentYear']<=1997)].copy()
    else: 
        df_trinagulo = df[df['GRCODE']== grcode].copy()
        
    df_g         = df_trinagulo.groupby(["AccidentYear", "DevelopmentLag"]).agg({'IncurLoss_F2': ['max']})
    df_g.columns = ['Pagos']
    df_g         = df_g.reset_index()
    pivot_data   = df_g.pivot(index='AccidentYear',columns='DevelopmentLag',values='Pagos').reset_index()
    pivot_data   = pivot_data.drop('AccidentYear', axis=1).cumsum(axis=1)
    
    return pivot_data


In [ ]:
input = pd.read_csv(os.path.normpath(os.getcwd() + os.sep + os.pardir)+"/data/ppauto_pos.csv")

input = input[input.DevelopmentYear <= 1997]

cleaning_cond = np.array(['Adriatic Ins Co', 'Aegis Grp', 'Agency Ins Co Of MD Inc',
       'Allegheny Cas Co', 'American Modern Ins Grp Inc',
       'Armed Forces Ins Exchange', 'Auto Club South Ins Co',
       'Baltica-Skandinavia Rein Co Of Amer', 'Bancinsure Inc',
       'Bell United Ins Co', 'Century-Natl Ins Co', 'Co-Operative Ins Co',
       'Consumers Ins Usa Inc', 'Cornerstone Natl Ins Co',
       'Federated Natl Ins Co', 'First Amer Ins Co',
       'Florists Mut Ins Grp', 'Harbor Ins Co', 'Homestead Ins Co',
       'Inland Mut Ins Co', 'Interstate Auto Ins Co Inc', 'Lancer Ins Co',
       'Lumber Ins Cos', 'Manhattan Re Ins Co', 'Mennonite Mut Ins Co',
       'Middle States Ins Co Inc', 'National Automotive Ins',
       'Nevada General Ins Co', 'New Jersey Citizens United Rcp Exch',
       'Nichido Fire & Marine Ins Co Ltd', 'Northwest Gf Mut Ins Co',
       'Ocean Harbor Cas Ins Co', 'Overseas Partners Us Reins Co',
       'Pacific Ind Ins Co', 'Pacific Pioneer Ins Co',
       'Pacific Specialty Ins Co', 'Penn Miller Grp',
       'Pennsylvania Mfg Asn Ins Co', 'Pioneer State Mut Ins Co',
       'Protective Ins Grp', 'San Antonio Reins Co',
       'Seminole Cas Ins Co', 'Southern Group Ind Inc',
       'Southern Mut Ins Co', 'Southland Lloyds Ins Co', 'Star Ins Grp',
       'Sterling Ins Co', 'Usauto Ins Co', 'Vanliner Ins Co',
       'Wea Prop & Cas Ins Co', 'Wellington Ins Co'])

input = input[~input.GRNAME.isin(cleaning_cond)]

In [ ]:
df_data        = pd.read_csv('medmal_pos.csv')
df_trg_entreno = triangulo(df_data, grcode=669, entreno=True)
df_trg_prueba  = triangulo(df_data, grcode=669, entreno=False)
df_trg_entreno

In [ ]:
Y = matrix_y(df_trg_entreno)
X = matrix_X(df_trg_entreno)

Y_X          = pd.merge(Y, X, on='y_ii', how='inner')
data_entreno = Y_X[Y_X['Y'].notna()]
data_entreno = data_entreno.drop(['y_ii'], axis=1)
data_entreno.head()

In [ ]:
Y_prueba      = matrix_y(df_trg_prueba)                   
x_prueba      = Y_X[Y_X['Y'].isna()].drop(['Y'], axis=1)
data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
data_prueba=data_prueba_.drop(['y_ii'], axis=1)
y_ii = data_prueba_['y_ii']

In [ ]:
x_entreno = data_entreno.drop('Y', axis=1)  # Features
y_entreno = data_entreno['Y']  # Target variable
x_prueba  = data_prueba.drop('Y', axis=1)  # Features
y_prueba  = data_prueba['Y']  # Target variable


model = LinearRegression()
model.fit(x_entreno, y_entreno)

y_pred = model.predict(x_prueba)
mse = mean_squared_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
mse